In [1]:
import warnings
warnings.filterwarnings('ignore',category=RuntimeWarning)
import xarray as xr
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import glob,os,sys
from tqdm.auto import tqdm
import proplot as plot
import json,pickle
import dask.array as da
import gc
from sklearn.decomposition import PCA
sys.path.insert(1, '/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/2020_TC_CRF/dev/freddy0218/scikit/')
from tools import derive_var,read_and_proc,preproc_noensemble
from tools.mlr import mlr, maria_IO
from tools.preprocess import do_eof,preproc_maria,preproc_haiyan
%matplotlib inline
plot.rc.update({'figure.facecolor':'w','axes.labelweight':'ultralight',
                'tick.labelweight':'ultralight','gridminor.linestyle':'--','title.weight':'normal','linewidth':0.5})

/tmp/ipykernel_3190148/1796742808.py:20: ProplotWarning: rc setting 'linewidth' was renamed to 'meta.width' in version 0.8.
  plot.rc.update({'figure.facecolor':'w','axes.labelweight':'ultralight',


In [2]:
def Pipeline(case='Maria',TYPE='3D' or '2D',suffix=None,expstart=[25, 49, 85, 25]):
    
    def produce_Xy(PCAdict=None,maria_data=None,nummem=None,divider=None,EXP=None,storename=None):
        # Normalized time series (X)
        maria_pca_nonorml = maria_IO.input_output( dict1, None, ['u', 'v', 'w', 'qv','theta', 'heatsum', 'hdia', 'lw', 'sw'], nummem ).produce_timeseries( maria_data )
        maria_pca_norml = maria_IO.input_output( dict1, None, ['u', 'v', 'w', 'qv','theta', 'heatsum', 'hdia', 'lw', 'sw'], nummem ).normalize_timeseries( maria_pca_nonorml )
        # Back to experiments for splitting
        exp_pca_norml = maria_IO.input_output( dict1, None, ['u', 'v', 'w', 'qv','theta', 'heatsum', 'hdia', 'lw', 'sw'], nummem ).back_to_exp( maria_pca_norml, divider ) 
        exp_pca_nonorml = maria_IO.input_output( dict1, None, ['u', 'v', 'w', 'qv','theta', 'heatsum', 'hdia', 'lw', 'sw'], nummem ).back_to_exp( maria_pca_nonorml, divider )
        
        TEMPts = np.max(np.mean(np.sqrt(np.asarray(maria_data['v']).reshape(np.asarray(maria_data['v']).shape[0],39,360,200)[:,0,...]**2+\
                                        np.asarray(maria_data['u']).reshape(np.asarray(maria_data['u']).shape[0],39,360,200)[:,0,...]**2),axis=1),axis=1)
        exp_pca_outputsTS = maria_IO.input_output( dict1, None, ['u', 'v', 'w', 'rh','theta', 'hdia', 'rad', 'lw', 'sw'], nummem )._back_to_exp( TEMPts, divider )
        print(len(exp_pca_outputsTS))
        X_dict = {}
        for ind,obj in enumerate(storename):
            test1 = (maria_IO.input_output(dict1,None,['u','v','w','qv','theta','heatsum','hdia','lw','sw'],None).make_X_nosep( exp_pca_norml, EXP[ind]))
            X_dict[obj] = test1
        yall = maria_IO.input_output( dict1, None, ['u', 'v', 'w', 'qv','theta', 'heatsum', 'hdia', 'lw', 'sw'], nummem ).make_Y_nosep( exp_pca_nonorml, np.linspace(0,35,36)+1)
        yall_ts = maria_IO.input_output( dict1, None, ['u', 'v', 'w', 'qv','theta', 'heatsum', 'hdia', 'lw', 'sw'], nummem ).make_Y_ts( exp_pca_outputsTS, np.linspace(0,35,36)+1)
        return X_dict,yall,yall_ts
    
    if case=='Maria':
        filepath = input('Enter Path:')
        #------------------------------------------------------------------------------------------------------------------------------------------
        # Import Preprocessed Flat Data and PCA dictionary
        #------------------------------------------------------------------------------------------------------------------------------------------
        if TYPE=='3D':
            tempdict = {}
            for varname in ['u','v','w','qv','theta','heatsum','hdia','lw','sw']:
                if varname=='lw':
                    temp = [read_and_proc.depickle(filepath+str(lime)+suffix+'_radcomp')['LW'] for lime in tqdm(['ctl','ncrf_36h','ncrf_60h','ncrf_96h','lwcrf'])]
                elif varname=='sw':
                    temp = [read_and_proc.depickle(filepath+str(lime)+suffix+'_radcomp')['SW'] for lime in tqdm(['ctl','ncrf_36h','ncrf_60h','ncrf_96h','lwcrf'])]
                else:
                    temp = [read_and_proc.depickle(filepath+str(lime)+suffix)[varname] for lime in tqdm(['ctl','ncrf_36h','ncrf_60h','ncrf_96h','lwcrf'])]
                tempdict[varname] = maria_IO.long_MariaExps(temp,expstart)[0]
        elif TYPE=='2D':
            tempdict = {}
            for varname in ['u','v','w','qv','theta','heatsum','hdia','lw','sw']:
                if varname=='lw':
                    temp = [read_and_proc.depickle(filepath+str(lime)+suffix+'_radcomp')['LW'] for lime in tqdm(['ctl','ncrf_36h','ncrf_60h','ncrf_96h','lwcrf'])]
                elif varname=='sw':
                    temp = [read_and_proc.depickle(filepath+str(lime)+suffix+'_radcomp')['SW'] for lime in tqdm(['ctl','ncrf_36h','ncrf_60h','ncrf_96h','lwcrf'])]
                else:
                    temp = [read_and_proc.depickle(filepath+str(lime)+suffix)[varname] for lime in tqdm(['ctl','ncrf_36h','ncrf_60h','ncrf_96h','lwcrf'])]
                tempdict[varname] = maria_IO.long_MariaExps(maria_IO.to_azim(temp),expstart)[0]            
        #------------------------------------------------------------------------------------------------------------------------------------------
        # Save to xarray
        #------------------------------------------------------------------------------------------------------------------------------------------
        dims = ['sample','flatarray']
        coords = dict(sample=np.linspace(0,tempdict['u'].shape[0]-1,tempdict['u'].shape[0]),flatarray=np.linspace(0,tempdict['u'].shape[1]-1,tempdict['u'].shape[1]))
        ds = xr.Dataset(coords=coords)
        maria_data=preproc_haiyan.build_a_xarray_dataset(ds=ds,varname=['u','v','w','theta','heatsum','hdia','lw','qv','sw'],\
                                                         varfile=[tempdict['u'],tempdict['v'],tempdict['w'],tempdict['theta'],tempdict['heatsum'],tempdict['hdia'],tempdict['lw'],tempdict['qv'],tempdict['sw']],dims=dims,coords=coords)
        del tempdict
        gc.collect()
        #------------------------------------------------------------------------------------------------------------------------------------------
        # Import PCA dictionary
        #------------------------------------------------------------------------------------------------------------------------------------------
        if TYPE=='3D':
            folderpath='/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/2020_TC_CRF/dev/freddy0218/testML/output/maria/processed/'
            dict1 = read_and_proc.depickle(folderpath+'PCA/PCAdict3D')
        elif TYPE=='2D':
            folderpath='/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/2020_TC_CRF/dev/freddy0218/testML/output/maria/processed/intermediate/'
            dict1 = read_and_proc.depickle(folderpath+'PCA/PCAdict')
            
        print("--Start producing Input / Output--")
        #------------------------------------------------------------------------------------------------------------------------------------------
        # Create Input/ output
        #------------------------------------------------------------------------------------------------------------------------------------------
        if TYPE=='3D':
            nummem=[54,26,50,5,75,12,12,10,10]
        elif TYPE=='2D':
            nummem = [20,11,20,5,12,7,7,5,5]
        EXP = [['lw','sw'],['lw','sw','hdia'],['lw','sw','hdia','qv'],['lw','sw','u','w'],['lw','sw','u','v','w'],['lw','sw','u','v','w','hdia'],
               ['u','v','w','theta','hdia','lw','sw'],['u','v','w','theta','qv','hdia','lw','sw']]
        storename = ['lwsw','lwswhdia','lwswhdiaqv','lwswuw','lwswuvw','lwswuvwhdia','lwswdtthuvw','lwswdtthuvwqv']
        #EXP = [['hdia','lw'],['theta','hdia','lw'],['u','v','w','theta','hdia','lw'],['u','v','theta','hdia','lw'],['u','v'],['u','v','w'],['u','v','hdia','lw'],['u','v','w','hdia','lw'],['u','v','w','theta','qv','hdia','lw']]
        #storename = ['dt','dtth','dtthuvw','dtthuv','uv','uvw','dtuv','dtuvw','dtthuvwqv']
        X_dict,yall,yall_ts = produce_Xy(dict1, maria_data, nummem, divider, EXP, storename)
        
        return maria_data, dict1, X_dict, yall, yall_ts

In [3]:
def Pipeline_decomp(case='Maria',TYPE='3D' or '2D',suffix=None,suffix_orig=None):
    
    if case=='Maria':
        filepath = input('Enter Path:')
        #------------------------------------------------------------------------------------------------------------------------------------------
        # Import Preprocessed Flat Data and PCA dictionary
        #------------------------------------------------------------------------------------------------------------------------------------------
        if TYPE=='3D':
            tempdict = {}
            for varname in ['lw']:
                temp = [read_and_proc.depickle(filepath+str(lime)+suffix_orig)[varname] for lime in tqdm(['ctl','ncrf_36h','ncrf_60h','ncrf_96h','lwcrf'])]
                tempdict[varname] = maria_IO.long_MariaExps(temp)[0]
            tempdict_decomp = {}
            for varname in ['LW', 'SW', 'LWC', 'SWC', 'swLW', 'swSW']:
                temp = [read_and_proc.depickle(filepath+str(lime)+suffix)[varname] for lime in tqdm(['ctl','ncrf_36h','ncrf_60h','ncrf_96h','lwcrf'])]
                tempdict_decomp[varname] = maria_IO.long_MariaExps(temp)[0]
                
        elif TYPE=='2D':
            tempdict = {}
            for varname in ['lw']:
                temp = [read_and_proc.depickle(filepath+str(lime)+suffix_orig)[varname] for lime in tqdm(['ctl','ncrf_36h','ncrf_60h','ncrf_96h','lwcrf'])]
                tempdict[varname] = maria_IO.long_MariaExps(maria_IO.to_azim(temp))[0]
            tempdict_decomp = {}
            for varname in ['LW', 'SW', 'LWC', 'SWC', 'swLW', 'swSW']:
                temp = [read_and_proc.depickle(filepath+str(lime)+suffix)[varname] for lime in tqdm(['ctl','ncrf_36h','ncrf_60h','ncrf_96h','lwcrf'])]
                tempdict_decomp[varname] = maria_IO.long_MariaExps(maria_IO.to_azim(temp))[0]          
        #------------------------------------------------------------------------------------------------------------------------------------------
        # Save to xarray
        #------------------------------------------------------------------------------------------------------------------------------------------
        dims = ['sample','flatarray']
        coords = dict(sample=np.linspace(0,tempdict['lw'].shape[0]-1,tempdict['lw'].shape[0]),flatarray=np.linspace(0,tempdict['lw'].shape[1]-1,tempdict['lw'].shape[1]))
        ds = xr.Dataset(coords=coords)
        maria_data_orig=preproc_haiyan.build_a_xarray_dataset(ds=ds,varname=['lw'],\
                                                         varfile=[tempdict['lw']],dims=dims,coords=coords)
        ds2 = xr.Dataset(coords=coords)
        maria_data_decomp=preproc_haiyan.build_a_xarray_dataset(ds=ds2,varname=['LW', 'SW', 'LWC', 'SWC', 'swLW', 'swSW'],\
                                                         varfile=[tempdict_decomp['LW'],tempdict_decomp['SW'],tempdict_decomp['LWC'],tempdict_decomp['SWC'],\
                                                                  tempdict_decomp['swLW'],tempdict_decomp['swSW']],dims=dims,coords=coords)
        del tempdict,tempdict_decomp
        gc.collect()
        #------------------------------------------------------------------------------------------------------------------------------------------
        # Import PCA dictionary
        #------------------------------------------------------------------------------------------------------------------------------------------
        if TYPE=='3D':
            folderpath='/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/2020_TC_CRF/dev/freddy0218/testML/output/maria/processed/'
            dict1 = read_and_proc.depickle(folderpath+'PCA/PCAdict2')
        elif TYPE=='2D':
            folderpath='/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/2020_TC_CRF/dev/freddy0218/testML/output/maria/processed/intermediate/'
            dict1 = read_and_proc.depickle(folderpath+'PCA/PCAdict')
        
        #print("--Start producing Input / Output--")
        #------------------------------------------------------------------------------------------------------------------------------------------
        # Create Input/ output
        #------------------------------------------------------------------------------------------------------------------------------------------
        if TYPE=='3D':
            nummem=[54,26,50,5,75,12,12,10,10]
        elif TYPE=='2D':
            nummem = [20,11,20,5,12,7,7,5,5]
            
        senTS = maria_IO.input_output( dict1, None, ['LW', 'SW', 'LWC', 'SWC', 'swLW', 'swSW'], nummem ).produce_Qsentimeseries(['LW', 'SW', 'LWC', 'SWC', 'swLW', 'swSW'],'lw', nummem[-1], maria_data_orig, maria_data_decomp)
        origTS = maria_IO.input_output( dict1, None, ['lw'], [5] ).produce_timeseries( maria_data_orig )
        senTS_norml,stdTS_norml = maria_IO.input_output( dict1, None, ['LW', 'SW', 'LWC', 'SWC', 'swLW', 'swSW'], nummem ).normalize_timeseries_decomp( senTS, origTS , ['LW', 'SW', 'LWC', 'SWC', 'swLW', 'swSW'], 'lw')
        #EXP = [['hdia','lw'],['theta','hdia','lw'],['u','v','w','theta','hdia','lw'],['u','v','theta','hdia','lw'],['u','v'],['u','v','w'],['u','v','hdia','lw'],['u','v','w','hdia','lw'],['u','v','w','theta','qv','hdia','lw']]
        #storename = ['dt','dtth','dtthuvw','dtthuv','uv','uvw','dtuv','dtuvw','dtthuvwqv']
        #X_dict,yall = produce_Xy(dict1, maria_data, nummem, divider, EXP, storename)
        
        return maria_data_orig, maria_data_decomp, dict1, senTS_norml, stdTS_norml

In [3]:
path = '/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/'
suffix = '_smooth_preproc_dict1b_g'
enter = '/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/2020_TC_CRF/dev/freddy0218/pca/output/uvwheat/preproc2/'

maria_u = [read_and_proc.depickle(path+'TCGphy/2020_TC_CRF/dev/freddy0218/pca/output/uvwheat/preproc2/'+str(lime)+suffix)['u'] for lime in tqdm(['ctl','ncrf_36h','ncrf_60h','ncrf_96h','lwcrf'])]
maria_v = [read_and_proc.depickle(path+'TCGphy/2020_TC_CRF/dev/freddy0218/pca/output/uvwheat/preproc2/'+str(lime)+suffix)['v'] for lime in tqdm(['ctl','ncrf_36h','ncrf_60h','ncrf_96h','lwcrf'])]
#maria_sw = [read_and_proc.depickle(path+'TCGphy/2020_TC_CRF/dev/freddy0218/pca/output/uvwheat/preproc2/'+str(lime)+suffix)['sw'] for lime in tqdm(['ctl','ncrf_36h','ncrf_60h','ncrf_96h','lwcrf'])]
#maria_qv = [read_and_proc.depickle(path+'TCGphy/2020_TC_CRF/dev/freddy0218/pca/output/uvwheat/preproc2/'+str(lime)+suffix)['qv'] for lime in tqdm(['ctl','ncrf_36h','ncrf_60h','ncrf_96h','lwcrf'])]
#25, 49, 85, 25
divider = np.asarray([maria_u[0][12:].shape[0],maria_u[1][25:].shape[0],maria_u[2][49:].shape[0],maria_u[3][85:].shape[0],maria_u[4][25:].shape[0]]).cumsum()

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

# Animation

In [99]:
wspd_ctrl = ((maria_u[0]**2+maria_v[0]**2)**(0.5)).reshape(maria_u[0].shape[0],39,360,200)
wspd_ncrf60 = ((maria_u[2]**2+maria_v[2]**2)**(0.5)).reshape(maria_u[2].shape[0],39,360,200)

In [119]:
import pyart
ANGLE = read_and_proc.azimuth2angle(np.linspace(0,359,360))
plot.rc.metacolor = 'k'
plot.rc.update({'axes.labelweight':'normal','tick.labelweight':'normal','figure.facecolor':'w','title.color':'k','fontname': 'Source Sans Pro', 'fontsize': 11,'linewidth':1.25})
plot.rc.axesfacecolor = 'w'
def normalize(array):
    return (array-np.nanmean(array))/np.nanstd(array)

for i in range(30,105):
    fig,axs = plot.subplots(ncols=2,nrows=1,axwidth=3.595,refaspect=1.031)
    aaaaa = axs[0].contourf(np.ma.masked_less(np.fliplr(np.flipud(read_and_proc.proc_tocart(wspd_ctrl.reshape(wspd_ctrl.shape[0],39,360,200)[i,0,...],ANGLE,True,False))),0.05),levels=np.linspace(0,30,61),cmap='pyart_Carbone17',extend='both')

    aaaab = axs[1].contourf(np.ma.masked_less(np.fliplr(np.flipud(read_and_proc.proc_tocart(wspd_ncrf36.reshape(wspd_ncrf36.shape[0],39,360,200)[i,0,...],ANGLE,True,False))),0.05),levels=np.linspace(0,30,61),cmap='pyart_Carbone17',extend='both')
    axs[0].colorbar(aaaaa,loc='b',label='Max. Surf. Winds (m s$^{-1}$)')
    axs[0].spines[['right', 'top','bottom','left']].set_visible(False)
    axs[0].set_xticks([])
    axs[0].set_xticks([], minor=True)
    axs[0].set_yticks([])
    axs[0].set_yticks([], minor=True)
    axs[0].format(title='CTRL',suptitle='Hr: '+str(i))
    axs[1].colorbar(aaaab,loc='b',label='Max. Surf. Winds (m s$^{-1}$)')
    axs[1].spines[['right', 'top','bottom','left']].set_visible(False)
    axs[1].set_xticks([])
    axs[1].set_xticks([], minor=True)
    axs[1].set_yticks([])
    axs[1].set_yticks([], minor=True)
    axs[1].format(title='NCRF_60')
    plt.savefig('./pic/'+str(i)+'.png',dpi=600)
    plt.close()
#axs[0].tick_params(top=False,left=False,bottom=False,right=False)
#plt.axis('off')
#plt.show()

/tmp/ipykernel_2413281/4151237851.py:3: ProplotWarning: rc setting 'linewidth' was renamed to 'meta.width' in version 0.8.
  plot.rc.update({'axes.labelweight':'normal','tick.labelweight':'normal','figure.facecolor':'w','title.color':'k','fontname': 'Source Sans Pro', 'fontsize': 11,'linewidth':1.25})
/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/miniconda3/lib/python3.9/site-packages/proplot/figure.py:1324: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  array = array.astype(np.int)
/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/miniconda3/li

In [125]:
from natsort import natsorted

In [127]:
# Create the frames
from PIL import Image
frames = []
imgs = natsorted(glob.glob("./pic/*.png"))
for i in imgs:
    new_frame = Image.open(i)
    frames.append(new_frame)
 
# Save into a GIF file that loops forever
frames[0].save('allgif.gif', format='GIF',
               append_images=frames[1:],
               save_all=True,
               duration=200, loop=0)

# Create input/output

In [4]:
import importlib
importlib.reload(maria_IO)

<module 'tools.mlr.maria_IO' from '/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/2020_TC_CRF/dev/freddy0218/scikit/tools/mlr/maria_IO.py'>

In [5]:
maria_data, dict1, X, y, y_ts = Pipeline('Maria','3D','_smooth_preproc_dict1b_g')
#TEMPts = np.max(np.mean(np.sqrt(maria_data['v'].data.reshape(maria_data['v'].data.shape[0],39,360,200)[:,0,...]**2+maria_data['u'].data.reshape(maria_data['u'].data.shape[0],39,360,200)[:,0,...]**2),axis=1),axis=1)

Enter Path: /work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/2020_TC_CRF/dev/freddy0218/pca/output/uvwheat/preproc2/


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

5


In [6]:
suffix='_smooth_preproc_dict1b_g_dudvdwdth'
dudvdwdth = {}
for ind,obj in enumerate(['du','dv','dw','dtheta']):
    array = [read_and_proc.depickle(path+'TCGphy/2020_TC_CRF/dev/freddy0218/pca/output/uvwheat/preproc2/'+str(lime)+suffix)[obj] for lime in tqdm(['ctl','ncrf_36h','ncrf_60h','ncrf_96h','lwcrf'])]
    tempn = [np.nanmean(array[0].reshape(144,39,360,200),axis=2).reshape(144,39*200),np.nanmean(array[1].reshape(144,39,360,200),axis=2).reshape(144,39*200),\
             np.nanmean(array[2].reshape(144,39,360,200),axis=2).reshape(144,39*200),np.nanmean(array[3].reshape(144,39,360,200),axis=2).reshape(144,39*200),\
            np.nanmean(array[4].reshape(144,39,360,200),axis=2).reshape(144,39*200)]
    temp = [tempn[0][12:,:],tempn[1][36:,:],tempn[2][60:,:],tempn[3][96:,:],tempn[4][36:,:]]
    dudvdwdth[obj] = np.concatenate(temp,axis=0)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [10]:
#datacheck(b,a,divider).dudvdwVAR(dudvdwdth,'w')
#[np.abs(b[obj].explained_variance_ratio_.cumsum()-0.91).argmin() for obj in ['u','v','w','qv','theta','heatsum','hdia','lw','sw']]

[7, 0, 7, 4, 2, 6, 6, 4, 4]

In [7]:
#nummem = [20,11,20,5,12,7,7,5,5]
nummem=[54,26,50,5,75,12,12,10,10]
X_dict_sep = maria_IO.input_output( dict1, None, ['u', 'v', 'w', 'qv','theta', 'heatsum', 'hdia', 'lw', 'sw'], nummem )._back_to_exp( X['lwswdtthuvw'], divider ) 
yall_sep = maria_IO.input_output( dict1, None, ['u', 'v', 'w', 'qv','theta', 'heatsum', 'hdia', 'lw', 'sw'], nummem )._back_to_exp( y[23], divider ) 
#yall_t_sep = maria_IO.input_output( dict1, None, ['u', 'v', 'w', 'qv','theta', 'heatsum', 'hdia', 'lw', 'sw'], nummem )._back_to_exp( np.asarray(y_ts[23]), divider ) 

In [9]:
def create_index(seed=None,explength=[133,109,85,49,109],percentage=None,findnum=49):
    rng = np.random.default_rng(seed)
    return [rng.choice(np.linspace(0,explength[i]-1,explength[i]),int(np.round(findnum*percentage[i])),replace=False) for i in range(5)]

def sep_train_valid(varlist=None,explength=[133,109,85,49,109]):
    def _out_index(length=None,index=None):
        return [int(obj) for obj in index], [int(obj) for obj in np.linspace(0,int(length-1),int(length)) if obj not in index]
    trainstore,validstore = [],[]
    for i in range(len(varlist)):
        validindex,trainindex = _out_index(explength[i],indices[i])
        validdata,traindata = varlist[i][validindex],varlist[i][trainindex]
        trainstore.append(traindata)
        validstore.append(validdata)
    return trainstore,validstore

In [20]:
# 3 done
Xtrain = np.concatenate([X_dict_sep[i] for i in [0,2,3]],axis=0)
Xvalid = (X_dict_sep[4])
Xtest = (X_dict_sep[1])
ytrain = (np.concatenate([y_ts[23][i] for i in [0,2,3]],axis=0))
yvalid = (y_ts[23][4])
ytest = y_ts[23][1]

In [14]:
def delete_padding(inTS=None,outTS=None):
    output_nozero,input_nozero = [],[]
    for i in range(len(outTS[:,0])):
        temp = outTS[i,:]
        tempin = inTS[i,:]
        if temp.all()==0:
            continue
        else:
            output_nozero.append(temp)
            input_nozero.append(tempin)
    return input_nozero,output_nozero

In [21]:
folderpath='/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/2020_TC_CRF/dev/freddy0218/testML/output/maria/processed/timeseries/'
read_and_proc.save_to_pickle(folderpath+'inputoutput2/Xtrain_24_val4',Xtrain,'PICKLE')
read_and_proc.save_to_pickle(folderpath+'inputoutput2/Xvalid_24_val4',Xvalid,'PICKLE')
read_and_proc.save_to_pickle(folderpath+'inputoutput2/Xtest_24_val4',Xtest,'PICKLE')
read_and_proc.save_to_pickle(folderpath+'inputoutput2/ytrain_24_val4',ytrain,'PICKLE')
read_and_proc.save_to_pickle(folderpath+'inputoutput2/yvalid_24_val4',yvalid,'PICKLE')
read_and_proc.save_to_pickle(folderpath+'inputoutput2/ytest_24_val4',ytest,'PICKLE')

In [25]:
np.asarray(delete_padding(Xtrain,ytrain)[0]).shape

(313, 237)

#### Decomposition

In [5]:
maria_data, maria_data_decomp, dict1, senTS_norml, stdTS_norml = Pipeline_decomp('Maria','2D','_smooth_preproc_dict1b_g_lwcomp','_smooth_preproc_dict1b_g')

Enter Path: /work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/2020_TC_CRF/dev/freddy0218/pca/output/uvwheat/preproc2/


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [8]:
nummem = [20,11,20,5,12,7,7,5,5]
X_dict_sep = maria_IO.input_output( dict1, None, ['u', 'v', 'w', 'qv','theta', 'heatsum', 'hdia', 'lw', 'sw'], nummem )._back_to_exp( senTS_norml['LW'], divider )

In [11]:
folderpath='/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/2020_TC_CRF/dev/freddy0218/testML/output/maria/processed/intermediate/'
read_and_proc.save_to_pickle(folderpath+'inputoutput/lwterms',senTS_norml,'PICKLE')
read_and_proc.save_to_pickle(folderpath+'inputoutput/stdterms',stdTS_norml,'PICKLE')